In [24]:
import json
import numpy as np
import requests 
import pandas as pd
from bs4 import BeautifulSoup as bs
from operator import itemgetter 
import time



In [33]:
RAW_DATA_FILEPATH = '/users/eric/PycharmProjects/testico/api_data.json'
URL_FILEPATH = '/users/eric/PycharmProjects/testico/ico_urls.txt'
TOKEN_FILE = '/users/eric/PycharmProjects/testico/token.csv'
FULLDATA_FILE = '/users/eric/PycharmProjects/testico/fulldata.csv'
NUMERICAL_DATA_FILE = '/users/eric/PycharmProjects/testico/full_num.csv'
TEXT_DATA_FILE = '/users/eric/PycharmProjects/testico/textfeature_dataframe.csv'
URL_COPY_FILEPATH = '/users/eric/PycharmProjects/testico/ico_urls copy.txt'
TOKEN_GIT_FEATURE_FILE = '/users/eric/PycharmProjects/testico/token_with_github_feature.csv'

In [26]:
def repolink_extract(url):
    try:
        html = requests.get(url)
    except Exception as e:
        print(e)
        print(url)
        return None
    htmltxt = bs(html.text, 'html.parser')
    
    
    repohtml = htmltxt.find_all('a', class_="d-inline-block", 
                                itemprop="name codeRepository")

    partial_repolink_list = []
    for item in repohtml:
        link = item.attrs['href']
        partial_repolink_list.append(link)
    return partial_repolink_list
    

    

def nextpage_extract(url):
    try:
        html = requests.get(url)
    except Exception as e:
        print(e)
        print(url)
        return None
    htmltxt = bs(html.text, 'html.parser')

    nextpage = htmltxt.find_all('a', 
                    class_="next_page",
                    rel="next")
    if len(nextpage) != 0:
        a = 'https://github.com'
        next_ = nextpage[0].attrs['href']
        nextlink = '{}{}'.format(a, next_)
        return nextlink
    else:
        return None


def repolink_extract_paginator(current_url):
    output = repolink_extract(current_url)
    next_url = nextpage_extract(current_url)
    while next_url is not None:
        current_url = next_url
        output_temp = repolink_extract(current_url)
        output.append(output_temp)
        next_url = nextpage_extract(current_url)
        time.sleep(1)
    repolink_list = []
    def reemoveNestings(l):
        for i in l:
            if type(i) == list:
                reemoveNestings(i)
            else:
                repolink_list.append(i)
    reemoveNestings(output)
    
    return repolink_list

def gitfeature_extract(partial_repolink):
    gitfeature_empty = {
            'watchers':0,
            'stars': 0,
            'forks':0, 
            'commits':0,
            'branches':0}
    a = 'https://github.com'
    full_repolink = '{}{}'.format(a, partial_repolink)
    try:
        githtml = requests.get(full_repolink)
    except Exception as e:
        print(e)
        print(partial_repolink)
        gitfeature = {
            'watchers':0,
            'stars': 0,
            'forks':0, 
            'commits':0,
            'branches':0}
        return gitfeature
    githtmltxt = bs(githtml.text, 'html.parser')
    otherfeature_html = githtmltxt.find_all(
       'span', class_="num text-emphasized")
    if len(otherfeature_html) == 0:
        
        return gitfeature_empty
    else:
        
        watch_html = githtmltxt.find_all(
            'a', 
            class_="social-count",
            href= '{}{}'.format(partial_repolink,'/watchers')
        )
        if len(watch_html) == 0:
            return gitfeature_empty
        else:

            watch = watch_html[0].text.strip().replace(',', '')
            stars_html = githtmltxt.find_all(
                'a',
                class_="social-count",
                href= '{}{}'.format(partial_repolink,'/stargazers')
            )
            stars = stars_html[0].text.strip().replace(',', '')
            forks_html = githtmltxt.find_all(
                'a',
                href= '{}{}'.format(partial_repolink,'/network/members'),
                class_="social-count"
            )

            forks = forks_html[0].text.strip().replace(',', '')

            otherfeature_html = githtmltxt.find_all(
               'span', class_="num text-emphasized")
            commits = otherfeature_html[0].text.strip().replace(',', '')
            branches = otherfeature_html[1].text.strip().replace(',', '')
            releases = otherfeature_html[2].text.strip().replace(',', '')
            gitfeature = {
                'watchers':watch,
                'stars': stars,
                'forks':forks, 
                'commits':commits,
                'branches':branches}
            return gitfeature
    


            

def repofeature_extract_paginator(current_url):
    output = repolink_extract(current_url)
    next_url = nextpage_extract(current_url)
    while next_url is not None:
        current_url = next_url
        output_temp = repolink_extract(current_url)
        output.append(output_temp)
        next_url = nextpage_extract(current_url)
        time.sleep(1)
    repolink_list = []
    def reemoveNestings(l):
        for i in l:
            if type(i) == list:
                reemoveNestings(i)
            else:
                repolink_list.append(i)
    reemoveNestings(output)
    d_list = []
    feature_sum = {}
    for partial_repolink in repolink_list:
        feature_dic = gitfeature_extract(partial_repolink)
        d_list.append(feature_dic)
    for d in d_list:
        for k in d.keys():
            feature_sum[k] = feature_sum.get(k, 0) + int((d[k]))
    return feature_sum

def gitfeature_html(github_account_html):
    if github_account_html is None:
        gitfeature = {
            'watchers':0,
            'stars': 0,
            'forks':0, 
            'commits':0,
            'branches':0}
        return gitfeature
    else:
        try:
            html = requests.get(github_account_html)
        except Exception as e:
            print(e)
            print(url)
            gitfeature ={
            'watchers':0,
            'stars': 0,
            'forks':0, 
            'commits':0,
            'branches':0}
            return gitfeature
        htmltxt = bs(html.text, 'html.parser')
        repohtml = htmltxt.find_all('a',class_="d-inline-block", 
                                    itemprop="name codeRepository")
        if len(repohtml) == 0:
            output = gitfeature_extract(github_account_html.replace('https://github.com', ''))
            return output


        else:
            output = repofeature_extract_paginator(github_account_html)
            return output
        
def get_webpage_link(html_body):

    html = html_body.find_all("div", class_="col-xs-6 col-md-6")
    if len(html) == 0:
        return None
    html2 = html[1]
    html3 = html2.contents
    website = [ii for ii in html3 if not ii == '\n'][0]
    weblink = website.get('href')
    return weblink


def get_github_link(html_body):

    html2 = html_body.find_all("div", class_="col-xs-6 col-md-9")
    if len(html2) == 0:
        return None
    icon = html2[0]
    icon1 = icon.contents
    link = [item for item in icon1 if not item == '\n']
    linktxt = []
    for xx in link:
        bb = xx.get('href')
        linktxt.append(bb)

    for xx in linktxt:
        if 'github' in xx:
            return xx

    return None

In [28]:
with open(URL_FILEPATH, 'r') as f:
    urls = f.readlines()

urls = [url.strip('\n') for url in urls]
results = []
for url in urls:
    html = requests.get(url)
    htmltxt = bs(html.text, 'html.parser')
    htmlbody = htmltxt.body
    heads = htmlbody.find_all("div", class_="col-xs-6 col-md-3")[2:]
    heads = [head.text.strip().split('\n')[0] for head in heads]
    datas = htmlbody.find_all("div", class_="col-xs-6 col-md-9")[2:]
    datas = [data.text.strip().split('\n')[0] for data in datas]
    ico_results = list(zip(heads, datas))


    github_link = get_github_link(htmlbody)
    website_link = get_webpage_link(htmlbody)
    ico_results.append(('Github URL', github_link))
    ico_results.append(('Official Website', website_link))
    
    
    git_feature = gitfeature_html(github_link)
    git_feature_list = []
    for key, value in git_feature.items():
        temp=(key, value)
        git_feature_list.append(temp)
        
    joinedlist = ico_results + git_feature_list
    results.append(joinedlist) 
    


final_results = []
for result in results:
    dict_result = {key: value for (key, value) in result}
    final_results.append(dict_result)

for idx, url in enumerate(urls):
    name = url.split('/')[-1]
    final_results[idx]['name'] = name
    final_results[idx]['url'] = url

df = pd.DataFrame(final_results)
# df.to_csv('token_with_github_feature.csv')

HTTPSConnectionPool(host='github.comhttps', port=443): Max retries exceeded with url: //www.github.com/expa%20%20nse-org (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10c11cf28>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
https://www.github.com/expa  nse-org
HTTPSConnectionPool(host='github.comhttp', port=443): Max retries exceeded with url: //www.github.com/cryptonetix (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10bd3a630>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
http://www.github.com/cryptonetix
HTTPSConnectionPool(host='github.comhttps', port=443): Max retries exceeded with url: //gist.github.com/Credits-com (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10c22cda0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not 

In [32]:
df.to_csv('token_with_github_feature.csv',index=False)

In [83]:
df

,Accepted Currencies,Coin Symbol,Country,Funding Goal,Funds in Escrow?,Github URL,Have Prototype?,Initial Token Price,Official Website,Platform,...,Restricted Countries,Total Raised,Total Tokens,branches,commits,forks,name,stars,url,watchers
0,NaN,REP,Unknown,NaN,No,https://github.com/AugurProject,NaN,$0.5800,https://www.augur.net/,Ethereum,...,United States,"$5,100,000",NaN,NaN,NaN,NaN,augur,NaN,https://icowatchlist.com/ico/augur,NaN
1,NaN,REP,Unknown,NaN,No,https://github.com/AugurProject,NaN,$0.5800,https://www.augur.net/,Ethereum,...,United States,"$5,100,000",NaN,644,84958,32284,lisk,1618,https://icowatchlist.com/ico/lisk,587
2,NaN,LSK,Germany,NaN,No,https://github.com/LiskHQ,NaN,$0.0747,https://lisk.io/,Not Available,...,United States,"$6,350,000",NaN,NaN,NaN,NaN,digixdao,NaN,https://icowatchlist.com/ico/digixdao,NaN
3,NaN,LSK,Germany,NaN,No,https://github.com/LiskHQ,NaN,$0.0747,https://lisk.io/,Not Available,...,United States,"$6,350,000",NaN,280,41535,2773,waves,3611,https://icowatchlist.com/ico/waves,641
4,NaN,DGD,Singapore,NaN,No,https://github.com/digixglobal,NaN,$3.2400,https://www.dgx.io/dgd/,Ethereum,...,United States,"$5,500,000",NaN,NaN,NaN,NaN,stratis,NaN,https://icowatchlist.com/ico/stratis,NaN
5,NaN,DGD,Singapore,NaN,No,https://github.com/digixglobal,NaN,$3.2400,https://www.dgx.io/dgd/,Ethereum,...,United States,"$5,500,000",NaN,711,29303,20191,taonetwork,655,https://icowatchlist.com/ico/taonetwork,323
6,NaN,WAVES,Unknown,NaN,No,https://github.com/WAVESPLATFORM/,NaN,$0.1880,https://wavesplatform.com/,Waves,...,United States,"$16,010,000",NaN,NaN,NaN,NaN,incent,NaN,https://icowatchlist.com/ico/incent,NaN
7,NaN,WAVES,Unknown,NaN,No,https://github.com/WAVESPLATFORM/,NaN,$0.1880,https://wavesplatform.com/,Waves,...,United States,"$16,010,000",NaN,702,52579,2548,neo,2243,https://icowatchlist.com/ico/neo,1135
8,NaN,STRAT,United Kingdom,NaN,No,None,NaN,$0.0073,https://stratisplatform.com/,Stratis,...,United States,"$590,000",NaN,NaN,NaN,NaN,blockpay,NaN,https://icowatchlist.com/ico/blockpay,NaN
9,NaN,STRAT,United Kingdom,NaN,No,None,NaN,$0.0073,https://stratisplatform.com/,Stratis,...,United States,"$590,000",NaN,0,0,0,firstblood,0,https://icowatchlist.com/ico/firstblood,0


In [34]:
scraped_data = pd.read_csv(TOKEN_GIT_FEATURE_FILE)
with open(RAW_DATA_FILEPATH, 'r') as f:
    api_data = json.load(f)
api_data = api_data['ico']['finished']
api_df = pd.DataFrame(api_data)

api_df.rename(columns={'all_time_roi': 'ROI', 'coin_symbol': 'Coin Symbol', 'icowatchlist_url': 'url'},
              inplace=True)
cols_to_use = api_df.columns.difference(scraped_data.columns)

fulldata_plus_git = pd.merge(scraped_data, api_df[cols_to_use], left_index=True, right_index=True, how='outer')
fulldata_plus_git.to_csv('fulldata_plus_git.csv', index=False)

In [35]:
NUMERICAL_DATA_PLUS_GIT_FILE = '/users/eric/PycharmProjects/testico/num_plus_git.csv'
full_num = pd.read_csv(NUMERICAL_DATA_PLUS_GIT_FILE, index_col=0)

In [36]:
full_num

,price_usd,Total days,watchers,stars,forks,branches,commits,Country_Australia,Country_Austria,Country_Canada,...,Platform_Ethereum Classic,Platform_Hyperledger,Platform_Maidsafe,Platform_NEO,Platform_Not Available,Platform_Rootstock,Platform_Stratis,Platform_Waves,binarize_Git,binary_ROI
Initial Token Price,,,,,,,,,,,,,,,,,,,,,
0.5800,19.21,45,590,1630,32289,644,85017,0,0,0,...,0,0,0,0,0,0,0,0,1,1
0.0747,1.83,28,656,3615,2777,283,41575,0,0,0,...,0,0,0,0,1,0,0,0,1,1
3.2400,36.92,0,324,655,20195,714,29437,0,0,0,...,0,0,0,0,0,0,0,0,1,1
0.1880,2.24,49,1135,2244,2549,703,52588,0,0,0,...,0,0,0,0,0,0,0,1,1,1
0.0073,0.75,35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
0.0470,0.14,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
0.1590,8.79,30,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
0.0692,0.10,1,36,138,1921,84,3559,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0.1200,0.19,32,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
